# __Web-Scraping my Fantasy Soccer Mini-league__

by Daniel De La Hormaza

1. Explanation of the Project
2. Gathering Data of the Mini-League's General Table
        - Scraping and Cleaning Data
        - Creating the Basic Dataframe
        - Populating the Basic Datafram with Team-Spcific Data
3. Gathering Data on Captain Decision-Making
        - Getting the 1140 Captain Selections
        - Playing with the Data
4. Recapping and Exporting Dataframes

This is a web scraping project about my Fantasy Soccer Mini-league, which is a game I have been playing with very close longtime friends for more than 10 years. This year I decided to level up my analysis of the game at the same time that I create this interesting web-scraping project.

If you are not familiar with fantasy sports, you have to select players in the game's platform. Your team performs based on the real-life performance of the chosen players. Sure, you can just select players and get luck; nevertheless, data analysis has a large impact on your performance.

For the past years, I haven't dedicated much time to the game. However, this is a great opportunity for a web-scraping project because data is all over the place. Scraping the data presented several challenges that this notebook tackles.

### The Biggest Challenge

I can separate the scraping project in 3 different stages that vary in difficulty. 

First, I scraped the General Table of my mini-league. This was a fairly easy scraping process that just required some data cleaning handled with the Pandas Library. 

Second, I designed a web-scraping program to visit each team's "history" site to grab data on how they were managed during the last season. 

Finally, I wanted information on how the teams chose their captain. Each team has to choose one captain each gameweek to score double that specific gameweek. Making the right decision can be the difference between a good or a bad gameweek. 

This third stage of the web-scraping program was the biggest challenge because it required the program to visit the 38 gameweeks for each of the 30 teams and grab the datapoint. It was a total of 1140 visits and data collections. Moreover, the selected captain can be a defender, a midfielder, a forward, or even the gaolkeeper. This means that the element to look for could be anywhere on the pitch, which is not ideal for a web-scraping project. Here is an example of how one team and the captain selected look for a specific gameweek:

<img src='Captain_Example.png'>

## __Let's start gathering data of the Mini-league's General Table:__

We start by importing several libraries that we have to use to gather data and manipulate it. We use BeautifulSoup to manipulate the HTML and Selenium to get the full HTML because BeatifulSoup has problems parsing it due to the presence of JavaScript code. 

In [6]:
import pandas as pd

from bs4 import BeautifulSoup as bs

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

import time, random

The General Table contains the standing of each team at the end of the season. Scraping this table is fairly easy but requires some cleaning of the data to get a basic dataframe that we can use in the future. 

The idea is to populate the basic dataframe with information gathered from each team's own site. 

In [99]:
# Selenium

driver = webdriver.Chrome(ChromeDriverManager().install())
url = "https://fantasy.premierleague.com/leagues/313426/standings/c?phase=1&page_new_entries=1&page_standings=1"
driver.get(url)
time.sleep(3) #pause needed for the data to load before parsing the HTML
soup = bs(driver.page_source, 'html.parser')
driver.quit()

[WDM] - Current google-chrome version is 84.0.4147
[WDM] - Get LATEST driver version for 84.0.4147


[WDM] - Get LATEST driver version for 84.0.4147
[WDM] - Trying to download new driver from http://chromedriver.storage.googleapis.com/84.0.4147.30/chromedriver_mac64.zip
[WDM] - Driver has been saved in cache [/Users/Hormaza/.wdm/drivers/chromedriver/mac64/84.0.4147.30]


In [8]:
# We now have the HTML in a BeautifulSoup element that we can use to get the data we need. 

#print(soup.prettify()) - This line allows us to check it but the result is very messy, so we'll keep it as a comment to preserve the order of the notebook

In [9]:
# Get the table data 

table_data = soup.find_all('td')

fpl_table = [] #list that contains the General Table
for i in table_data:
    fpl_table.append(i.text)

print(fpl_table)

['1', 'AtenasFC*Sergio Guerrero', '88', '2362', '2', 'Alpha United F.C.*Leonardo Avila', '78', '2302', '3', 'Yankianos SB*Ronald Castillo', '45', '2282', '4', 'ACM1PT F.C. - S.B.Mauricio Ferrandino', '66', '2278', '5', "lil Jose's TeamJosé Andrés Araya", '67', '2222', '6', 'Zizú FC - SBJosue Hernandez', '58', '2182', '7', 'ProVercelliJosé Andrés Rodríguez', '82', '2181', '8', 'lomitocrFrancisco Fonseca', '66', '2160', '9', 'Latin Lovers - SBKenneth Espinoza', '89', '2144', '10', 'Lazio FC - S.BUlises Dixon', '64', '2141', '11', 'Arsenal89 F.F.C.Nazim Hosein', '72', '2139', '12', 'KwiatFCErin Kwiatkoski', '57', '2115', '13', 'Toros FCJorge Calderón', '51', '2109', '14', 'ManUtd94Mauricio Picado', '45', '2095', '15', 'PURPLE MONSTERCarlos Roldán', '85', '2080', '16', 'Koperos SB*Jose Munoz', '82', '2043', '17', 'Real MadridDaniel Hormaza', '82', '2041', '18', 'New Age FCIgnacio Avila', '44', '2026', '19', 'The SquadLaura Mata', '75', '2021', '20', 'Mambo Kingz F.C.Sergio Vargas', '60', '

In [10]:
# Slice the Ranking column from the list

rank = fpl_table[::4]
print(rank)

['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30']


In [11]:
# Slice the Total Points column

tot_pts = fpl_table[3::4]
print(tot_pts)

['2362', '2302', '2282', '2278', '2222', '2182', '2181', '2160', '2144', '2141', '2139', '2115', '2109', '2095', '2080', '2043', '2041', '2026', '2021', '1987', '1968', '1953', '1946', '1942', '1920', '1914', '1911', '1904', '1847', '1781']


In [12]:
# The team names are joined with the player names in the table data list. Thus, we used other HTML element to gent only the team names. 

teams_html = soup.find_all('strong')

teams = []
for i in teams_html:
    teams.append(i.text)

#The result is generating one last item that it's a team. Slice the list to contain only temas. 
teams = teams[0:-1]
print(teams)

['AtenasFC*', 'Alpha United F.C.*', 'Yankianos SB*', 'ACM1PT F.C. - S.B.', "lil Jose's Team", 'Zizú FC - SB', 'ProVercelli', 'lomitocr', 'Latin Lovers - SB', 'Lazio FC - S.B', 'Arsenal89 F.F.C.', 'KwiatFC', 'Toros FC', 'ManUtd94', 'PURPLE MONSTER', 'Koperos SB*', 'Real Madrid', 'New Age FC', 'The Squad', 'Mambo Kingz F.C.', 'MJS-SB', 'Muerte a LDA', 'Coto 47 F.C', 'Yozco - SB', 'Colchoneros', 'GoBlues FC', 'Las Águilas SB', 'Killers FC', 'Zanetti 4 Ever', 'Inter**']


### Create Dataframe

In [13]:
fpl_df = pd.DataFrame()

fpl_df['Rank'] = rank
fpl_df['Team'] = teams
fpl_df['Tot_PTS'] = tot_pts

fpl_df.head()

,Rank,Team,Tot_PTS
0,1,AtenasFC*,2362
1,2,Alpha United F.C.*,2302
2,3,Yankianos SB*,2282
3,4,ACM1PT F.C. - S.B.,2278
4,5,lil Jose's Team,2222


### Gather data from each team

In [14]:
# We need each team's id to construct the URLs to loop over and get data from each team. 

ids_finder = [links['href'] for links in soup.find_all(class_="Link-a4a9pd-1 hnWgWg")]

ids = [i.split('/') for i in ids_finder]

ids = [i[2] for i in ids]

print(ids)

['1008002', '1872185', '725578', '584561', '134325', '410532', '2923052', '1878021', '1848172', '1307980', '25883', '2880924', '4350786', '968906', '67031', '490074', '3276470', '1318543', '193118', '384777', '3266133', '1316024', '4224823', '4024167', '1378577', '4572548', '1574497', '811284', '1783689', '4510354']


In [15]:
# let's add these IDs to the dataframe to know the corresponding team of each ID. 

fpl_df['IDs'] = ids

fpl_df.head()

,Rank,Team,Tot_PTS,IDs
0,1,AtenasFC*,2362,1008002
1,2,Alpha United F.C.*,2302,1872185
2,3,Yankianos SB*,2282,725578
3,4,ACM1PT F.C. - S.B.,2278,584561
4,5,lil Jose's Team,2222,134325


In [16]:
base_urls = 'https://fantasy.premierleague.com/entry/'

In [17]:
# Generate a list of URLs to loop over when web-scrapin each team data

list_urls = []

for i in ids:
    list_urls.append(base_urls + i + "/history")
    
print(list_urls[0:5])

# Every team has a "history" section with data of the whole season. This table contains very insightful information of how was the team managed.
# The list of URLs created go to each team's history table. 

['https://fantasy.premierleague.com/entry/1008002/history', 'https://fantasy.premierleague.com/entry/1872185/history', 'https://fantasy.premierleague.com/entry/725578/history', 'https://fantasy.premierleague.com/entry/584561/history', 'https://fantasy.premierleague.com/entry/134325/history']


#### Start scraping Atenas FC's data

I started by scraping Atenas FC's data to analyze the output and HTML elements. Then I can use what I learn from the data to create a web-scraping strategy for each team. 

In [18]:
driver = webdriver.Chrome(ChromeDriverManager().install())
url = list_urls[0]
driver.get(url)
time.sleep(3)
soup2 = bs(driver.page_source, 'html.parser')
driver.quit()

[WDM] - Current google-chrome version is 84.0.4147
[WDM] - Get LATEST driver version for 84.0.4147
[WDM] - Driver [/Users/Hormaza/.wdm/drivers/chromedriver/mac64/84.0.4147.30/chromedriver] found in cache


In [19]:
# Get total transfers made during the season and the final team value. These data is part of the history table but it's quicker to grab them from the team's general information. 

transf = soup2.find_all(class_ = "Entry__DataListValue-sc-1kf863-3 cmkvzv")[4]
transf = transf.text
print('Total Transfers:')
print(transf)
print('------------------')

value = soup2.find_all(class_ = "Entry__DataListValue-sc-1kf863-3 cmkvzv")[5]
value = value.text
value = float(value.replace('£',''))
print('Final Team Value:')
print(value)

Total Transfers:
64
------------------
Final Team Value:
104.3


In [20]:
# Let's grab the data from the history table

hist_table = [i.text for i in soup2.find_all('td')] # This creates a list of all the data contained in the history table. We will slice this list to get the specific observations we want. 

In [21]:
# Grab the points left in the bench each gameweek and add them up to get the total bench points at the end of the season. 

tb_gw = hist_table[2:470:10]

tb_gw = [int(i) for i in tb_gw]

bench_points = sum(tb_gw)

print('Total Bench Points:')
print(bench_points)

Total Bench Points:
222


In [22]:
# Grab points lost in transfers each gameweek and add them up to get the total transfer points lost at the end of the season. 

tch_gw = hist_table[5:475:10]

tch_gw = [int(i) for i in tch_gw]

transf_points = sum(tch_gw)

print('Total Transfer Points:')
print(transf_points)

Total Transfer Points:
68


In [23]:
# We want to alse get each gameweek's accumulated points to create a separate dataframe later on, which will give us a good visualization of the season. 

points_accum = hist_table[6:475:10]

points_accum = [int(i) for i in points_accum]

# Eliminate duplicated created by the Covid-19 suspension of the season. Create a set as a way to eliminate duplicates and transform it back to a list. 

points_accum = list(set(points_accum))

points_accum.sort(reverse=False)

print(len(points_accum)) # 38 is the right number of gameweeks
print(points_accum)

38
[65, 116, 151, 222, 288, 350, 398, 440, 474, 528, 600, 656, 730, 775, 829, 895, 961, 1017, 1087, 1150, 1187, 1242, 1303, 1384, 1454, 1528, 1600, 1639, 1696, 1792, 1886, 1948, 2035, 2093, 2154, 2186, 2278, 2362]


#### Now we can loop over each team's history URL to get the team-specific data

In [24]:
# Generate lists that will be appended by the scraping program 

total_transf = []

team_value = []

bench_points = []

transf_points = []

points_accum = [] #This will be a list of lists. 

for i in list_urls:
    #loop over the links
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.get(i)
    time.sleep(5)
    soup3 = bs(driver.page_source, 'html.parser')
    driver.quit()
    
    #Grab Total Transfers
    transf = soup3.find_all(class_ = "Entry__DataListValue-sc-1kf863-3 cmkvzv")[4]
    total_transf.append(transf.text)
    
    # Grab Team Value
    value = soup3.find_all(class_ = "Entry__DataListValue-sc-1kf863-3 cmkvzv")[5]
    value = value.text
    team_value.append(value)
    
    # Creat List of the history table data
    hist_table = [td.text for td in soup3.find_all('td')]
    
    #Grab total bench points
    tb_gw = hist_table[2:470:10]
    tb_gw = [int(bench) for bench in tb_gw]
    bench_points.append(sum(tb_gw))
    
    #Grab total transfer points
    tch_gw = hist_table[5:475:10]
    tch_gw = [int(transf) for transf in tch_gw]
    transf_points.append(sum(tch_gw))
    
    
    #List to create accumulated points 
    points = hist_table[6:475:10]
    points = [int(point) for point in points]
    points = list(set(points))
    points.sort(reverse=False)
    points_accum.append(points)
    
    
    time.sleep(1+2*random.random()) # Make the requests appear random

[WDM] - Current google-chrome version is 84.0.4147
[WDM] - Get LATEST driver version for 84.0.4147
[WDM] - Driver [/Users/Hormaza/.wdm/drivers/chromedriver/mac64/84.0.4147.30/chromedriver] found in cache


[WDM] - Current google-chrome version is 84.0.4147
[WDM] - Get LATEST driver version for 84.0.4147
[WDM] - Driver [/Users/Hormaza/.wdm/drivers/chromedriver/mac64/84.0.4147.30/chromedriver] found in cache


[WDM] - Current google-chrome version is 84.0.4147
[WDM] - Get LATEST driver version for 84.0.4147
[WDM] - Driver [/Users/Hormaza/.wdm/drivers/chromedriver/mac64/84.0.4147.30/chromedriver] found in cache


[WDM] - Current google-chrome version is 84.0.4147
[WDM] - Get LATEST driver version for 84.0.4147


[WDM] - Driver [/Users/Hormaza/.wdm/drivers/chromedriver/mac64/84.0.4147.30/chromedriver] found in cache
[WDM] - Current google-chrome version is 84.0.4147
[WDM] - Get LATEST driver version for 84.0.4147
[WDM] - Driver [/Users/Hormaza/.wdm/drivers/chromedriver/mac64/84.0.4147.30/chromedriver] found in cache


...

[WDM] - Current google-chrome version is 84.0.4147
[WDM] - Get LATEST driver version for 84.0.4147
[WDM] - Driver [/Users/Hormaza/.wdm/drivers/chromedriver/mac64/84.0.4147.30/chromedriver] found in cache


[WDM] - Current google-chrome version is 84.0.4147
[WDM] - Get LATEST driver version for 84.0.4147
[WDM] - Driver [/Users/Hormaza/.wdm/drivers/chromedriver/mac64/84.0.4147.30/chromedriver] found in cache


[WDM] - Current google-chrome version is 84.0.4147
[WDM] - Get LATEST driver version for 84.0.4147
[WDM] - Driver [/Users/Hormaza/.wdm/drivers/chromedriver/mac64/84.0.4147.30/chromedriver] found in cache


[WDM] - Current google-chrome version is 84.0.4147
[WDM] - Get LATEST driver version for 84.0.4147
[WDM] - Driver [/Users/Hormaza/.wdm/drivers/chromedriver/mac64/84.0.4147.30/chromedriver] found in cache


[WDM] - Current google-chrome version is 84.0.4147
[WDM] - Get LATEST driver version for 84.0.4147
[WDM] - Driver [/Users/Hormaza/.wdm/drivers/chromedriver/mac64/84.0.4147.30/chromedriver] found in cache


In [25]:
print(transf_points)

[68, 12, 60, 56, 32, 20, 28, 48, 64, 52, 32, 16, 48, 40, 108, 52, 32, 16, 0, 64, 12, 24, 28, 20, 52, 52, 16, 0, 4, 8]


In [26]:
print(bench_points)

[222, 210, 315, 173, 217, 245, 359, 210, 203, 185, 180, 285, 284, 327, 270, 164, 256, 257, 229, 220, 360, 257, 398, 159, 392, 227, 161, 262, 138, 102]


In [27]:
print(total_transf)

['64', '37', '59', '44', '46', '39', '61', '52', '50', '49', '50', '62', '39', '47', '60', '55', '54', '46', '36', '54', '25', '39', '31', '30', '48', '28', '12', '7', '16', '7']


In [28]:
print(team_value)

['£104.3', '£102.1', '£102.2', '£97.3', '£103.7', '£103.4', '£102.1', '£104.4', '£105.5', '£102.9', '£101.6', '£100.2', '£99.1', '£100.1', '£102.2', '£101.1', '£98.6', '£100.2', '£98.7', '£102.0', '£100.7', '£98.4', '£99.5', '£98.6', '£99.4', '£98.6', '£98.5', '£99.1', '£98.0', '£95.8']


In [29]:
# Now let's add these lists to our dataframe

fpl_df['Total_Transf'] = total_transf

fpl_df['Final_Val'] = team_value

fpl_df['Trans_Points'] = transf_points

fpl_df['Bench_Points'] = bench_points

fpl_df.head()

,Rank,Team,Tot_PTS,IDs,Total_Transf,Final_Val,Trans_Points,Bench_Points
0,1,AtenasFC*,2362,1008002,64,£104.3,68,222
1,2,Alpha United F.C.*,2302,1872185,37,£102.1,12,210
2,3,Yankianos SB*,2282,725578,59,£102.2,60,315
3,4,ACM1PT F.C. - S.B.,2278,584561,44,£97.3,56,173
4,5,lil Jose's Team,2222,134325,46,£103.7,32,217


In [30]:
# Change data types

fpl_df['Rank'] = pd.to_numeric(fpl_df['Rank'])

fpl_df['Tot_PTS'] = pd.to_numeric(fpl_df['Tot_PTS'])

fpl_df['Total_Transf'] = pd.to_numeric(fpl_df['Total_Transf'])

fpl_df.columns = ['Rank', 'Team', 'Total_Points','ID', 'Transfers', 'Final_Value', 'Transf_Points', 'Bench_Points']

In [31]:
fpl_df.head()

,Rank,Team,Total_Points,ID,Transfers,Final_Value,Transf_Points,Bench_Points
0,1,AtenasFC*,2362,1008002,64,£104.3,68,222
1,2,Alpha United F.C.*,2302,1872185,37,£102.1,12,210
2,3,Yankianos SB*,2282,725578,59,£102.2,60,315
3,4,ACM1PT F.C. - S.B.,2278,584561,44,£97.3,56,173
4,5,lil Jose's Team,2222,134325,46,£103.7,32,217


#### Let's also see how the top 5 teams performed during the season

We will generate a dataframe with the accumulated score of the top 5 teams of the mini-league. This dataframe will be used in another notebook with visualuzations of the data scraped in here. 

In [39]:
points_acuum_top5 = points_accum[0:5]

top5_df = pd.DataFrame(points_acuum_top5)

top5_df = top5_df.transpose()

top5_df.columns = fpl_df['Team'][0:5]

range_list = ['1', '2', '3', '4' , '5', '6' , '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18','19' ,'20' , '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38']

GW = []
for i in range_list:
    GW.append('GW ' + i)
    
top5_df['GW'] = GW

top5_df = top5_df.set_index('GW')

top5_df.head()

Team,AtenasFC*,Alpha United F.C.*,Yankianos SB*,ACM1PT F.C. - S.B.,lil Jose's Team
GW,,,,,
GW 1,65,68,67,96,92
GW 2,116,111,135,155,142
GW 3,151,149,179,196,166
GW 4,222,226,252,257,221
GW 5,288,283,318,306,273


## Captain Selection

We want to analyze Atenas FC's decision-making when choosing the captains. We'll have to deal with the Covid-19 suspension creating a URL list that skips those gameweeks. 

Another issue to handle is that some captains chosen are now injured and that adds an extra element to the HTML.

In [40]:
# Create list with urls

range_list_1_29 = ['1', '2', '3', '4' , '5', '6' , '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18','19' ,'20' , '21', '22', '23', '24', '25', '26', '27', '28', '29']

range_list_39_47 = ['39', '40', '41', '42', '43', '44', '45', '46', '47']

Atenas_GW = []

base_gw_url = "https://fantasy.premierleague.com/entry/1008002/event/"

for i in range_list_1_29:
    Atenas_GW.append(base_gw_url + i)
    
for i in range_list_39_47:
    Atenas_GW.append(base_gw_url + i)


print(len(Atenas_GW)) # We should have 38 URLs, one per gameweek.

38


In [41]:
# Get the captain for one gameweek to define the web-scraping strategy

driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(Atenas_GW[3]) # this gameweek Atenas FC chose a captain that is now injured
time.sleep(4)
soup3 = bs(driver.page_source, 'html.parser')
driver.quit()

[WDM] - Current google-chrome version is 84.0.4147
[WDM] - Get LATEST driver version for 84.0.4147
[WDM] - Driver [/Users/Hormaza/.wdm/drivers/chromedriver/mac64/84.0.4147.30/chromedriver] found in cache


In [42]:
# We use the "C" element that we see on the pitch next to the captain. This is the only element that would help us get a path to the captain name and points.

# This code accesses the element with injured captains

soup3.find(class_="TeamPitchElement__StyledCaptain-sc-202u14-1 jUhfsE").previous.previous.previous.previous.previous.previous.previous.previous.previous.previous.previous.previous

'Agüero'

In [43]:
# This code accesses injured captain's points

soup3.find(class_="TeamPitchElement__StyledCaptain-sc-202u14-1 jUhfsE").previous.previous.previous.previous.previous.previous.previous.previous.previous.previous

'32'

In [44]:
# Now look for the path to a not injured captain

driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(Atenas_GW[0]) 
time.sleep(4)
soup3 = bs(driver.page_source, 'html.parser')
driver.quit()

[WDM] - Current google-chrome version is 84.0.4147
[WDM] - Get LATEST driver version for 84.0.4147
[WDM] - Driver [/Users/Hormaza/.wdm/drivers/chromedriver/mac64/84.0.4147.30/chromedriver] found in cache


In [45]:
# Path to the captain's name

soup3.find(class_="TeamPitchElement__StyledCaptain-sc-202u14-1 jUhfsE").previous.previous.previous.previous.previous.previous.previous.previous.previous.previous

'Salah'

In [46]:
# Path to the captain's points

cap_points = soup3.find(class_="TeamPitchElement__StyledCaptain-sc-202u14-1 jUhfsE").previous.previous.previous.previous.previous.previous.previous.previous

cap_points = int(cap_points)

print(cap_points)

24


#### Scraping all the Captains

We have the path to get both injured and not injured captains. We will control the flow of the program to get either of the options. 

We will loop over the list of URLs to get the captain of each gameweek. We will ask the program to get the points made by the captain first. Then it will have to convert the result into an "int" variable. We now that if the captain was injured the program would access an element different than the points, so trying to convert them into integers would generate an error. We will use that error to control the flow of the program with a "try and except" clause. 

Teams have also the chance to use a "triple captain chip" (TC) once per season. The TC makes your captain score triple rather than double. For the sake of comparability, we'll exclude the TC from our analysis. However, our code has to find the TC to then drop it when we have our dataframe. 

In [51]:
Captains = []

Captains_Points = []

for i in Atenas_GW:

    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.get(i)
    time.sleep(5)
    soup3 = bs(driver.page_source, 'html.parser')
    driver.quit()
    
    try:
        points_path = soup3.find(class_="TeamPitchElement__StyledCaptain-sc-202u14-1 jUhfsE").previous.previous.previous.previous.previous.previous.previous.previous
        points_path = int(points_path)
        Captains_Points.append(points_path)
    
        captain_path = soup3.find(class_="TeamPitchElement__StyledCaptain-sc-202u14-1 jUhfsE").previous.previous.previous.previous.previous.previous.previous.previous.previous.previous
        Captains.append(captain_path)
    
    except TypeError:
        points_path = soup3.find(class_="TeamPitchElement__StyledCaptain-sc-202u14-1 jUhfsE").previous.previous.previous.previous.previous.previous.previous.previous.previous.previous
        points_path = int(points_path)
        Captains_Points.append(points_path)
    
        captain_path = soup3.find(class_="TeamPitchElement__StyledCaptain-sc-202u14-1 jUhfsE").previous.previous.previous.previous.previous.previous.previous.previous.previous.previous.previous.previous
        Captains.append(captain_path)
    
    except AttributeError: # This is the error we get when manager used the "Triple Captain Chip"
        
        points_tc = "TC_points"
        Captains_Points.append(points_tc)
        
        captain_tc = 'TC'
        Captains.append(captain_tc)

    
    time.sleep(1+2*random.random())

[WDM] - Current google-chrome version is 84.0.4147
[WDM] - Get LATEST driver version for 84.0.4147


[WDM] - Driver [/Users/Hormaza/.wdm/drivers/chromedriver/mac64/84.0.4147.30/chromedriver] found in cache
[WDM] - Current google-chrome version is 84.0.4147
[WDM] - Get LATEST driver version for 84.0.4147
[WDM] - Driver [/Users/Hormaza/.wdm/drivers/chromedriver/mac64/84.0.4147.30/chromedriver] found in cache


[WDM] - Current google-chrome version is 84.0.4147
[WDM] - Get LATEST driver version for 84.0.4147
[WDM] - Driver [/Users/Hormaza/.wdm/drivers/chromedriver/mac64/84.0.4147.30/chromedriver] found in cache


[WDM] - Current google-chrome version is 84.0.4147
[WDM] - Get LATEST driver version for 84.0.4147
[WDM] - Driver [/Users/Hormaza/.wdm/drivers/chromedriver/mac64/84.0.4147.30/chromedriver] found in cache


[WDM] - Current google-chrome version is 84.0.4147
[WDM] - Get LATEST driver version for 84.0.4147
[WDM] - Driver [/Users/Hormaza/.wdm/drivers/chromedriver/mac64/84.0.4147.30/chromedriver] found in cache


...

[WDM] - Current google-chrome version is 84.0.4147
[WDM] - Get LATEST driver version for 84.0.4147
[WDM] - Driver [/Users/Hormaza/.wdm/drivers/chromedriver/mac64/84.0.4147.30/chromedriver] found in cache


[WDM] - Current google-chrome version is 84.0.4147
[WDM] - Get LATEST driver version for 84.0.4147
[WDM] - Driver [/Users/Hormaza/.wdm/drivers/chromedriver/mac64/84.0.4147.30/chromedriver] found in cache


[WDM] - Current google-chrome version is 84.0.4147
[WDM] - Get LATEST driver version for 84.0.4147
[WDM] - Driver [/Users/Hormaza/.wdm/drivers/chromedriver/mac64/84.0.4147.30/chromedriver] found in cache


[WDM] - Current google-chrome version is 84.0.4147
[WDM] - Get LATEST driver version for 84.0.4147
[WDM] - Driver [/Users/Hormaza/.wdm/drivers/chromedriver/mac64/84.0.4147.30/chromedriver] found in cache


[WDM] - Current google-chrome version is 84.0.4147
[WDM] - Get LATEST driver version for 84.0.4147
[WDM] - Driver [/Users/Hormaza/.wdm/drivers/chromedriver/mac64/84.0.4147.30/chromedriver] found in cache


In [52]:
print(Captains)

['Salah', 'Salah', 'Kane', 'Agüero', 'Agüero', 'Agüero', 'Agüero', 'Agüero', 'Abraham', 'De Bruyne', 'Agüero', 'Kane', 'Vardy', 'Mané', 'Vardy', 'Mané', 'Mané', 'Jiménez', 'Kane', 'De Bruyne', 'Vardy', 'Vardy', 'De Bruyne', 'Salah', 'Salah', 'Salah', 'Salah', 'Salah', 'Aubameyang', 'De Bruyne', 'De Bruyne', 'Martial', 'Fernandes', 'De Bruyne', 'Fernandes', 'Sterling', 'Martial', 'TC']


In [53]:
print(Captains_Points)

[24, 6, 4, 32, 12, 18, 8, 4, 4, 12, 12, 4, 24, 4, 10, 0, 12, 14, 16, 28, 0, 10, 4, 32, 32, 6, 14, 4, 4, 30, 16, 6, 30, 14, 10, 2, 10, 'TC_points']


#### Scraping all the Captains of the Mini-League

We have been understanding how the Fantasy Premier League's website is built to get to this point. This is the more complex part of the web-scraping tool that we have been building. 

We'll make our program to visit a total of 1140 URLs (38 gameweeks for each of the 30 teams), grab the captain and its points, and store the data in lists that we'll use to create a dataframe.

In [54]:
# Create list with urls

League_GW = []

for i in ids:
    range_list_1_29 = ['1', '2', '3', '4' , '5', '6' , '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18','19' ,'20' , '21', '22', '23', '24', '25', '26', '27', '28', '29']
    range_list_39_47 = ['39', '40', '41', '42', '43', '44', '45', '46', '47']

    Team_GW = []

    base_gw_url = "https://fantasy.premierleague.com/entry/"

    for gw in range_list_1_29:
        Team_GW.append(base_gw_url + i + "/event/" + gw)
    
    for gw in range_list_39_47:
        Team_GW.append(base_gw_url + i + "/event/" + gw)
    
    League_GW.append(Team_GW)

print(League_GW[0]) # Len() should be 30. There are 30 lists with 38 gameweeks.

['https://fantasy.premierleague.com/entry/1008002/event/1', 'https://fantasy.premierleague.com/entry/1008002/event/2', 'https://fantasy.premierleague.com/entry/1008002/event/3', 'https://fantasy.premierleague.com/entry/1008002/event/4', 'https://fantasy.premierleague.com/entry/1008002/event/5', 'https://fantasy.premierleague.com/entry/1008002/event/6', 'https://fantasy.premierleague.com/entry/1008002/event/7', 'https://fantasy.premierleague.com/entry/1008002/event/8', 'https://fantasy.premierleague.com/entry/1008002/event/9', 'https://fantasy.premierleague.com/entry/1008002/event/10', 'https://fantasy.premierleague.com/entry/1008002/event/11', 'https://fantasy.premierleague.com/entry/1008002/event/12', 'https://fantasy.premierleague.com/entry/1008002/event/13', 'https://fantasy.premierleague.com/entry/1008002/event/14', 'https://fantasy.premierleague.com/entry/1008002/event/15', 'https://fantasy.premierleague.com/entry/1008002/event/16', 'https://fantasy.premierleague.com/entry/1008002

In [55]:
Captains = []

Captains_Points = []

for i in League_GW:
    
    for gw in i:
        driver = webdriver.Chrome(ChromeDriverManager().install())
        driver.get(gw)
        time.sleep(5)
        soup4 = bs(driver.page_source, 'html.parser')
        driver.quit()
    
        try:
            points_path = soup4.find(class_="TeamPitchElement__StyledCaptain-sc-202u14-1 jUhfsE").previous.previous.previous.previous.previous.previous.previous.previous
            points_path = int(points_path)
            Captains_Points.append(points_path)
    
            captain_path = soup4.find(class_="TeamPitchElement__StyledCaptain-sc-202u14-1 jUhfsE").previous.previous.previous.previous.previous.previous.previous.previous.previous.previous
            Captains.append(captain_path)
    
        except TypeError: # This exception handles the impossibility to convert the result of the "try" clause into an int variable.
            try:
                points_path = soup4.find(class_="TeamPitchElement__StyledCaptain-sc-202u14-1 jUhfsE").previous.previous.previous.previous.previous.previous.previous.previous.previous.previous
                points_path = int(points_path)
                Captains_Points.append(points_path)
    
                captain_path = soup4.find(class_="TeamPitchElement__StyledCaptain-sc-202u14-1 jUhfsE").previous.previous.previous.previous.previous.previous.previous.previous.previous.previous.previous.previous
                Captains.append(captain_path)
        
            except: # This exception is for cases where neither the captain nor the vice-captain played, so the website shows a "-" that can't be converted into a int variable.
                points_error = 0
                Captains_Points.append(points_error)
    
                captain_error = "No Cap"
                Captains.append(captain_error)
    
        except ValueError: # Same as the previouse exception but for the non-injured captains.
            points_error = 0
            Captains_Points.append(points_error)
    
            captain_error = "No Cap"
            Captains.append(captain_error)
        
        except AttributeError: # This is the error we get when manager used the "Triple Captain Chip"
        
            points_tc = "TC_points"
            Captains_Points.append(points_tc)
        
            captain_tc = 'TC'
            Captains.append(captain_tc)

    
        time.sleep(1+2*random.random())
    

print("Captains and Points obtained successfully")

[WDM] - Current google-chrome version is 84.0.4147
[WDM] - Get LATEST driver version for 84.0.4147


[WDM] - Get LATEST driver version for 84.0.4147
[WDM] - Trying to download new driver from http://chromedriver.storage.googleapis.com/84.0.4147.30/chromedriver_mac64.zip
[WDM] - Driver has been saved in cache [/Users/Hormaza/.wdm/drivers/chromedriver/mac64/84.0.4147.30]
[WDM] - Current google-chrome version is 84.0.4147
[WDM] - Get LATEST driver version for 84.0.4147
[WDM] - Driver [/Users/Hormaza/.wdm/drivers/chromedriver/mac64/84.0.4147.30/chromedriver] found in cache


[WDM] - Current google-chrome version is 84.0.4147
[WDM] - Get LATEST driver version for 84.0.4147
[WDM] - Driver [/Users/Hormaza/.wdm/drivers/chromedriver/mac64/84.0.4147.30/chromedriver] found in cache


[WDM] - Current google-chrome version is 84.0.4147
[WDM] - Get LATEST driver version for 84.0.4147
[WDM] - Driver [/Users/Hormaza/.wdm/drivers/chromedriver/mac64/84.0.4147.30/chromedriver] found in cache


[WDM] - Current google-chrome version is 84.0.4147
[WDM] - Get LATEST driver version for 84.0.4147
[WDM] - Driver [/Users/Hormaza/.wdm/drivers/chromedriver/mac64/84.0.4147.30/chromedriver] found in cache


...

[WDM] - Current google-chrome version is 84.0.4147
[WDM] - Get LATEST driver version for 84.0.4147
[WDM] - Driver [/Users/Hormaza/.wdm/drivers/chromedriver/mac64/84.0.4147.30/chromedriver] found in cache


[WDM] - Current google-chrome version is 84.0.4147
[WDM] - Get LATEST driver version for 84.0.4147
[WDM] - Driver [/Users/Hormaza/.wdm/drivers/chromedriver/mac64/84.0.4147.30/chromedriver] found in cache


[WDM] - Current google-chrome version is 84.0.4147
[WDM] - Get LATEST driver version for 84.0.4147
[WDM] - Driver [/Users/Hormaza/.wdm/drivers/chromedriver/mac64/84.0.4147.30/chromedriver] found in cache


[WDM] - Current google-chrome version is 84.0.4147
[WDM] - Get LATEST driver version for 84.0.4147
[WDM] - Driver [/Users/Hormaza/.wdm/drivers/chromedriver/mac64/84.0.4147.30/chromedriver] found in cache


 
Captains and Points obtained successfully


In [56]:
print(Captains[0:38])

['Salah', 'Salah', 'Kane', 'Agüero', 'Agüero', 'Agüero', 'Agüero', 'Agüero', 'Abraham', 'De Bruyne', 'Agüero', 'Kane', 'Vardy', 'Mané', 'Vardy', 'Mané', 'Mané', 'Jiménez', 'Kane', 'De Bruyne', 'Vardy', 'Vardy', 'De Bruyne', 'Salah', 'Salah', 'Salah', 'Salah', 'Salah', 'Aubameyang', 'De Bruyne', 'De Bruyne', 'Martial', 'Fernandes', 'De Bruyne', 'Fernandes', 'Sterling', 'Martial', 'TC']


In [57]:
print(Captains_Points[0:38])

[24, 6, 4, 32, 12, 18, 8, 4, 4, 12, 12, 4, 24, 4, 10, 0, 12, 14, 16, 28, 0, 10, 4, 32, 32, 6, 14, 4, 4, 30, 16, 6, 30, 14, 10, 2, 10, 'TC_points']


In [58]:
# Grouping the lits of captains into sublists to then create the dataframe

import itertools

def grouper(n, it):
    it = iter(it)
    return iter(lambda: list(itertools.islice(it, n)), [])

Captains_List = list(grouper(38, Captains)) 

In [60]:
# Creating the dataframe of all captains chosen by each team. 

captains_df = pd.DataFrame(Captains_List)

In [61]:
captains_df = captains_df.transpose()

In [62]:
captains_df.head()

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,Salah,Kane,Salah,Salah,Salah,Kane,Salah,Salah,Salah,Salah,...,Fraser,Salah,Salah,King,Kane,Vardy,Aubameyang,Firmino,Kane,Firmino
1,Salah,De Bruyne,Aubameyang,Salah,Salah,Aubameyang,Salah,Salah,Salah,Salah,...,Fraser,Salah,Sterling,King,Aubameyang,Salah,Aubameyang,Salah,Mané,Firmino
2,Kane,De Bruyne,Martial,Kane,Kane,Kane,Sterling,Sterling,Sterling,Kane,...,Sterling,Sterling,Kane,Kane,Kane,Salah,Rashford,Salah,Kane,Firmino
3,Agüero,De Bruyne,Salah,Salah,Sterling,Sterling,Sterling,Sterling,Sterling,Sterling,...,Sterling,Sterling,Sterling,Sterling,Bernardo Silva,Salah,Sterling,Salah,Sterling,Firmino
4,Agüero,Kane,Salah,Sterling,Salah,Agüero,Sterling,Salah,Sterling,Agüero,...,Sterling,Salah,Kane,Salah,Kane,Salah,Agüero,Salah,Mané,Firmino


In [63]:
captains_df.columns = fpl_df['Team']

In [64]:
captains_df.head()

Team,AtenasFC*,Alpha United F.C.*,Yankianos SB*,ACM1PT F.C. - S.B.,lil Jose's Team,Zizú FC - SB,ProVercelli,lomitocr,Latin Lovers - SB,Lazio FC - S.B,...,MJS-SB,Muerte a LDA,Coto 47 F.C,Yozco - SB,Colchoneros,GoBlues FC,Las Águilas SB,Killers FC,Zanetti 4 Ever,Inter**
0,Salah,Kane,Salah,Salah,Salah,Kane,Salah,Salah,Salah,Salah,...,Fraser,Salah,Salah,King,Kane,Vardy,Aubameyang,Firmino,Kane,Firmino
1,Salah,De Bruyne,Aubameyang,Salah,Salah,Aubameyang,Salah,Salah,Salah,Salah,...,Fraser,Salah,Sterling,King,Aubameyang,Salah,Aubameyang,Salah,Mané,Firmino
2,Kane,De Bruyne,Martial,Kane,Kane,Kane,Sterling,Sterling,Sterling,Kane,...,Sterling,Sterling,Kane,Kane,Kane,Salah,Rashford,Salah,Kane,Firmino
3,Agüero,De Bruyne,Salah,Salah,Sterling,Sterling,Sterling,Sterling,Sterling,Sterling,...,Sterling,Sterling,Sterling,Sterling,Bernardo Silva,Salah,Sterling,Salah,Sterling,Firmino
4,Agüero,Kane,Salah,Sterling,Salah,Agüero,Sterling,Salah,Sterling,Agüero,...,Sterling,Salah,Kane,Salah,Kane,Salah,Agüero,Salah,Mané,Firmino


In [65]:
# Grouping the points generated by captains to create a dataframe

Captains_Points_List = list(grouper(38, Captains_Points)) 

In [66]:
captains_points_df = pd.DataFrame(Captains_Points_List)

captains_points_df = captains_points_df.transpose()

captains_points_df.columns = fpl_df['Team']

captains_points_df.head()

Team,AtenasFC*,Alpha United F.C.*,Yankianos SB*,ACM1PT F.C. - S.B.,lil Jose's Team,Zizú FC - SB,ProVercelli,lomitocr,Latin Lovers - SB,Lazio FC - S.B,...,MJS-SB,Muerte a LDA,Coto 47 F.C,Yozco - SB,Colchoneros,GoBlues FC,Las Águilas SB,Killers FC,Zanetti 4 Ever,Inter**
0,24,26,24,24,24,26,24,24,24,24,...,2,24,24,4,26,4,12,10,26,10
1,6,22,12,6,6,12,6,6,6,6,...,4,6,12,12,12,6,12,6,26,16
2,4,10,10,4,4,4,16,16,16,4,...,16,16,4,4,4,30,0,30,4,4
3,32,26,12,12,6,6,6,6,6,6,...,6,6,6,6,12,12,6,12,6,24
4,12,10,18,4,18,12,4,18,4,12,...,4,18,10,18,10,18,12,18,30,16


### Let's play with the data!

It would be ideal to export these dataframes into a SQL database and just make queries to visualize the data. However, we'll keep this notebook simple and construct one dataframe showing Atenas FC's captain selection.

In [68]:
range_list = ['1', '2', '3', '4' , '5', '6' , '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18','19' ,'20' , '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38']

GW = []
for i in range_list:
    GW.append('GW ' + i)

In [69]:
Atenas_Cap_df = pd.DataFrame()

Atenas_Cap_df['GW'] = GW

Atenas_Cap_df['Captains'] = captains_df['AtenasFC*']

Atenas_Cap_df['Captains_Points'] = captains_points_df['AtenasFC*']


In [71]:
Atenas_Cap_df.head()

,GW,Captains,Captains_Points
0,GW 1,Salah,24
1,GW 2,Salah,6
2,GW 3,Kane,4
3,GW 4,Agüero,32
4,GW 5,Agüero,12


In [72]:
# Let's see how Atenas FC distributed its captains

Atenas_Cap_df.groupby(['Captains']).count()

,GW,Captains_Points
Captains,,
Abraham,1,1
Agüero,6,6
Aubameyang,1,1
De Bruyne,6,6
Fernandes,2,2
Jiménez,1,1
Kane,3,3
Mané,3,3
Martial,2,2


In [73]:
# Now let's see how many times Atenas FC made a good captain decision

Atenas_Good_Cap = Atenas_Cap_df.drop(37)

Atenas_Good_Cap.dtypes

GW                 object
Captains           object
Captains_Points    object
dtype: object

In [74]:
Atenas_Good_Cap['Captains_Points'] = pd.to_numeric(Atenas_Good_Cap['Captains_Points'])

Atenas_Good_Cap['Good_Cap'] = (Atenas_Good_Cap['Captains_Points'] > 7)

Atenas_Good_Cap.groupby(['Good_Cap']).count()

# This number does not include the triple captain because it is very easy for a triple captain to get more than 7 points.

,GW,Captains,Captains_Points
Good_Cap,,,
False,14,14,14
True,23,23,23


## Let's Recap and Save!

We have the following dataframes that we'll save as a CSV file to use them in another visualization notebook:

In [47]:
# General Tables

fpl_df.to_csv("Tabla_General.csv", index=False)

top5_df.to_csv("Top5.csv", index=True)

In [76]:
# Captains Tables

captains_df.to_csv("Captains.csv", index=False)

captains_points_df.to_csv("Captains_Points.csv", index=False)

In [91]:
Captains_df2 = pd.DataFrame(Captains)

In [93]:
Caps_Tot_df = pd.DataFrame(Captains_df2[0].value_counts())

In [96]:
Caps_Tot_df.columns = ['Caps']

Caps_Tot_df.head()

,Caps
Salah,180
Vardy,155
De Bruyne,115
Agüero,98
Mané,96


In [98]:
Caps_Tot_df.to_csv("Captains_Total.csv", index=True)